# Lab 9: Automated Annotation

**Module 9** - CellTypist and SingleR

## Objectives
- Run automated cell type annotation
- Compare automated vs manual results
- Assess annotation confidence


In [ ]:
import scanpy as sc
import numpy as np
import matplotlib.pyplot as plt

# Note: Install celltypist first: pip install celltypist
try:
    import celltypist
    from celltypist import models
    CELLTYPIST_AVAILABLE = True
except ImportError:
    CELLTYPIST_AVAILABLE = False
    print("CellTypist not installed. Run: pip install celltypist")

sc.settings.set_figure_params(dpi=100, facecolor='white')

# Load data
adata = sc.datasets.pbmc3k_processed()
print(f"Cells: {adata.n_obs}")


In [ ]:
if CELLTYPIST_AVAILABLE:
    # Download models (first time only)
    # models.download_models()
    
    # List available models
    print("Available CellTypist models:")
    print(models.models_description().head())


In [ ]:
if CELLTYPIST_AVAILABLE:
    # Run CellTypist annotation
    model = models.Model.load(model='Immune_All_Low.pkl')
    
    predictions = celltypist.annotate(adata, model=model, majority_voting=True)
    
    # Add predictions to adata
    adata = predictions.to_adata()
    
    # View predictions
    print("\nCellTypist predictions:")
    print(adata.obs['predicted_labels'].value_counts())


In [ ]:
if CELLTYPIST_AVAILABLE:
    # Compare automated vs original clusters
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    sc.pl.umap(adata, color='louvain', ax=axes[0], show=False, title='Original clusters')
    sc.pl.umap(adata, color='majority_voting', ax=axes[1], show=False, title='CellTypist')
    
    plt.tight_layout()
    plt.show()


In [ ]:
if CELLTYPIST_AVAILABLE:
    # Check confidence scores
    sc.pl.umap(adata, color='conf_score', cmap='viridis', title='Annotation confidence')
    
    # Low confidence cells may need manual review
    low_conf = adata.obs['conf_score'] < 0.5
    print(f"Low confidence cells: {low_conf.sum()} ({100*low_conf.mean():.1f}%)")
